In [84]:
"""Mathematics part of Support Vector Machine is very easy ,So just go through any link online here we are only focussed on coding stuff"""
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [85]:
data=pd.read_csv('train.csv')
Sample=pd.read_csv('test.csv')

In [86]:
data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [87]:
data.columns#just getting all the column names

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [88]:
data.isna().sum()#just checking how many null values have in each column

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [89]:
Sample.isna().sum()#how many null values in our final testing dataset

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [90]:
#from observation we are going to remove some columns from our dataset
features=data[['Pclass','Sex','SibSp','Parch','Fare','Embarked']].copy()
target=data['Survived'].copy()

In [91]:
features.describe()#Find nothing useful

,Pclass,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000
mean,2.308642,0.523008,0.381594,32.204208
std,0.836071,1.102743,0.806057,49.693429
min,1.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,7.910400
50%,3.000000,0.000000,0.000000,14.454200
75%,3.000000,1.000000,0.000000,31.000000
max,3.000000,8.000000,6.000000,512.329200


In [92]:
features['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [93]:
#here we can easily see that the relative frequency of S is far greater than other two, So we can replace the 2 Nan values with S
features['Embarked']=features['Embarked'].replace(np.NaN,'S')

In [94]:
features['Embarked'].isna().sum()

0

In [95]:
li=list(set(features['Embarked']))#make the list of all different values in Embarked
gi=list(set(features['Sex']))#make the list of all different values in Sex
print(li,'\n',gi)

#now we are going to replace it with the numbers, So that our model can do some work
features['Embarked']=features['Embarked'].replace(li,[0,1,2])
features['Sex']=features['Sex'].replace(gi,[0,1])

['Q', 'S', 'C'] 
 ['male', 'female']


In [97]:
features.head()

,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,3,0,1,0,7.2500,1
1,1,1,1,0,71.2833,2
2,3,1,0,0,7.9250,1
3,1,1,1,0,53.1000,1
4,3,0,0,0,8.0500,1


In [98]:
features.describe()

,Pclass,Sex,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,0.352413,0.523008,0.381594,32.204208,1.102132
std,0.836071,0.477990,1.102743,0.806057,49.693429,0.514624
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,0.000000,7.910400,1.000000
50%,3.000000,0.000000,0.000000,0.000000,14.454200,1.000000
75%,3.000000,1.000000,1.000000,0.000000,31.000000,1.000000
max,3.000000,1.000000,8.000000,6.000000,512.329200,2.000000


In [99]:
#going to apply maximum value normalization
features=features/features.max()
features.describe()

,Pclass,Sex,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.769547,0.352413,0.065376,0.063599,0.062858,0.551066
std,0.278690,0.477990,0.137843,0.134343,0.096995,0.257312
min,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.666667,0.000000,0.000000,0.000000,0.015440,0.500000
50%,1.000000,0.000000,0.000000,0.000000,0.028213,0.500000
75%,1.000000,1.000000,0.125000,0.000000,0.060508,0.500000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#  Model Preparation

In [100]:
#Now our dataset is ready it is time to see SVM in action

x_train,x_test,y_train,y_test=train_test_split(features,target,test_size=0.2)
print(x_train.shape,' ',x_test.shape,' ',y_train.shape,' ',y_test.shape)

(712, 6)   (179, 6)   (712,)   (179,)


In [101]:
clf=SVC(gamma='auto')#gamma set to auto so that it can automatically set the most preferrable kernel

In [102]:
clf.fit(x_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [103]:
y_train_pred=clf.predict(x_train)
train_score = clf.score(x_train,y_train)

In [104]:
train_score

0.7823033707865169

In [105]:
def accuracy(y_pred,y):
    correct=len(y_pred[y_pred==y])
    #print(correct," ",len(y.T))
    return correct/len(y)*100
print(accuracy(y_train_pred,y_train))

78.23033707865169


In [106]:
test_score=clf.score(x_test,y_test)

In [107]:
test_score

0.8044692737430168

# Now we are going to play with kernels and find out which suits best

In [108]:
clf_rbf=SVC(kernel='rbf')
clf_rbf.fit(x_train,y_train)
score=clf_rbf.score(x_train,y_train)
score1=clf_rbf.score(x_test,y_test)
print('training_ Score: ',score," testing_score: ",score1)

training_ Score:  0.7823033707865169  testing_score:  0.8044692737430168


E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [109]:
clf_li=SVC(kernel='linear')
clf_li.fit(x_train,y_train)
score=clf_li.score(x_train,y_train)
score1=clf_li.score(x_test,y_test)
print('training_ Score: ',score," testing_score: ",score1)

training_ Score:  0.7823033707865169  testing_score:  0.8044692737430168


In [110]:
clf_poly=SVC(kernel='poly')
clf_poly.fit(x_train,y_train)
score=clf_poly.score(x_train,y_train)
score1=clf_poly.score(x_test,y_test)
print('training_ Score: ',score," testing_score: ",score1)

training_ Score:  0.7808988764044944  testing_score:  0.8044692737430168


E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


# Now we are going to play with C hyperparameter and try to find out perfect c

### What is C?

In [120]:
#The C parameter tells the SVM optimization how much you want to avoid misclassifying each training example. For large values of C, the optimization will choose a smaller-margin hyperplane if that hyperplane does a better job of getting all the training points classified correctly. Conversely, a very small value of C will cause the optimizer to look for a larger-margin separating hyperplane, even if that hyperplane misclassifies more points. For very tiny values of C, you should get misclassified examples, often even if your training data is linearly separable.

In [130]:
for i in range(1,26):
    clf=SVC(kernel='rbf',C=10*i)
    clf.fit(x_train,y_train)
    score=clf.score(x_train,y_train)
    score1=clf.score(x_test,y_test)
    print( 10*i,' training_ Score: ',score," testing_score: ",score1)
    

E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change f

10  training_ Score:  0.7907303370786517  testing_score:  0.8268156424581006
20  training_ Score:  0.800561797752809  testing_score:  0.8435754189944135
30  training_ Score:  0.8033707865168539  testing_score:  0.8435754189944135
40  training_ Score:  0.8047752808988764  testing_score:  0.8435754189944135
50  training_ Score:  0.8061797752808989  testing_score:  0.8435754189944135
60  training_ Score:  0.8061797752808989  testing_score:  0.8435754189944135
70  training_ Score:  0.8061797752808989  testing_score:  0.8379888268156425
80  training_ Score:  0.8047752808988764  testing_score:  0.8435754189944135


E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change f

90  training_ Score:  0.8061797752808989  testing_score:  0.8379888268156425
100  training_ Score:  0.8033707865168539  testing_score:  0.8379888268156425
110  training_ Score:  0.8075842696629213  testing_score:  0.8435754189944135
120  training_ Score:  0.8075842696629213  testing_score:  0.8435754189944135
130  training_ Score:  0.8103932584269663  testing_score:  0.8435754189944135
140  training_ Score:  0.8103932584269663  testing_score:  0.8435754189944135
150  training_ Score:  0.8103932584269663  testing_score:  0.8435754189944135


E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change f

160  training_ Score:  0.8103932584269663  testing_score:  0.8435754189944135
170  training_ Score:  0.8103932584269663  testing_score:  0.8435754189944135
180  training_ Score:  0.8103932584269663  testing_score:  0.8435754189944135
190  training_ Score:  0.8117977528089888  testing_score:  0.8435754189944135
200  training_ Score:  0.8075842696629213  testing_score:  0.8379888268156425
210  training_ Score:  0.8075842696629213  testing_score:  0.8379888268156425
220  training_ Score:  0.8075842696629213  testing_score:  0.8379888268156425
230  training_ Score:  0.8075842696629213  testing_score:  0.8379888268156425
240  training_ Score:  0.8075842696629213  testing_score:  0.8379888268156425
250  training_ Score:  0.8075842696629213  testing_score:  0.8379888268156425


E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change f

In [111]:
#score on both is approxiamately same which means that their is no overfitting and under fitting
pred_features=Sample[['Pclass','Sex','SibSp','Parch','Fare','Embarked']].copy()

In [112]:
pred_features.isna().sum()

Pclass      0
Sex         0
SibSp       0
Parch       0
Fare        1
Embarked    0
dtype: int64

In [113]:
pred_features['Fare']=pred_features['Fare'].replace(np.NaN, np.mean(pred_features['Fare']))
pred_features['Embarked']=pred_features['Embarked'].replace(li,[0,1,2])
pred_features['Sex']=pred_features['Sex'].replace(gi,[0,1])

In [114]:
pred_features.isna().sum()

Pclass      0
Sex         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [115]:
pred_features.head(10)

,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,3,0,0,0,7.8292,0
1,3,1,1,0,7.0000,1
2,2,0,0,0,9.6875,0
3,3,0,0,0,8.6625,1
4,3,1,1,1,12.2875,1
5,3,0,0,0,9.2250,1
6,3,1,0,0,7.6292,0
7,2,0,1,1,29.0000,1
8,3,1,0,0,7.2292,2
9,3,0,2,0,24.1500,1


In [116]:
pred_features=pred_features/pred_features.max()#maximum value normalization

In [131]:
clf=SVC(kernel='rbf', C=190)
clf.fit(features,target)#training our model on whole data set
final_res=clf.predict(pred_features)

E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [132]:
len(final_res)

418

In [133]:
output = pd.DataFrame({'PassengerId': Sample.PassengerId,
                       'Survived': final_res})
output.to_csv('Titanic_result.csv', index=False)